# Research hackathon year 1 - Fraudulent job advertisement detection

Data: http://emscad.samos.aegean.gr/

Aim: Engineering structured data to predict fraudulent job advertisement.

### Imports

In [1]:
import pandas as pd
import numpy as np

# For feature engineering
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk.tokenize as nt
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thaole/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/thaole/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load data
raw_data = pd.read_csv("emscad_v1.csv")
raw_data.head(3)

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"<h3>We're Food52, and we've created a groundbr...","<p>Food52, a fast-growing, James Beard Award-w...",<ul>\r\n<li>Experience with content management...,NaN,f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"<h3>90 Seconds, the worlds Cloud Video Product...",<p>Organised - Focused - Vibrant - Awesome!<br...,<p><b>What we expect from you:</b></p>\r\n<p>Y...,<h3><b>What you will get from us</b></h3>\r\n<...,f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,<h3></h3>\r\n<p>Valor Services provides Workfo...,"<p>Our client, located in Houston, is actively...",<ul>\r\n<li>Implement pre-commissioning and co...,NaN,f,t,f,NaN,NaN,NaN,NaN,NaN,f,f


## Data Cleaning

In [4]:
#Function to return wordcounts in strings
def clean_sequence(string):
    string = string.encode("ascii", "ignore").decode()
    string_words = word_tokenize(string)
    return len([word for word in string_words if word.lower() not in stop_words])

def clean_data(data):
    #Change all boolean variables to binary representations.
    data['fraudulent'] = np.where(data['fraudulent']=='t', 1, 0)
    data['telecommuting'] = np.where(data['telecommuting']=='t', 1, 0)
    data['has_company_logo'] = np.where(data['has_company_logo']=='t', 1, 0)
    data['has_questions'] = np.where(data['has_questions']=='t', 1, 0)
    
    #Get wordcounts
    data['description'] = data['description'].apply(lambda x: clean_sequence(str(x)))
    data['benefits'] = data['benefits'].apply(lambda x: clean_sequence(str(x)))
    data['requirements'] = data['requirements'].apply(lambda x: clean_sequence(str(x)))
    data['company_profile'] = data['company_profile'].apply(lambda x: clean_sequence(str(x)))
    return data


def change_to_ordinal(data):
    scale_mapper = {'Entry level':1, 'Not Applicable':0, 'Associate':3, 'Internship':2, 'Mid-Senior level':4, 'Director':5,'Executive':5}
    data['required_experience 1'] = data['required_experience'].replace(scale_mapper)
    data['required_experience 1'] = data['required_experience 1'].replace(np.nan, 0, regex=True)
    return data

list1 = ['..','!!','??']
spamwords_fraudulent = ['home', 'extra' 'easy money', 'week', 'online', 'no experience']

#Function to check whether a text has consecutive puntuation.
def consecutive_punct(list1, list2):
    if any( str in list1 for str in list2):
        return 1
    return 0

In [5]:
cleaned_data= clean_data(raw_data)
cleaned_data

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,137,155,170,1,0,1,0,Other,Internship,NaN,NaN,Marketing,0,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,272,317,344,253,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,142,42,223,1,0,1,0,NaN,NaN,NaN,NaN,NaN,0,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,85,423,272,222,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,230,395,210,9,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,367,176,201,158,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0,f
17876,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,287,227,173,249,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0,f
17877,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,30,292,285,1,0,0,0,Full-time,NaN,NaN,NaN,NaN,0,f
17878,Graphic Designer,"NG, LA, Lagos",NaN,NaN,1,70,101,37,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0,f


## Feature Engineering

In [6]:
# requiring low education
cleaned_data['low_required_education'] = 0
cleaned_data.loc[cleaned_data['required_education'] == ( 'High School or equivalent' or 'Unspecified' or 'Some High School Coursework' or '' or 'Vocational - HS Diploma' or 'Vocational - Degree' or 'Some College Coursework Completed'), ['low_required_education']] = 1

In [7]:
cleaned_data['low_required_education'].value_counts()

0    15800
1     2080
Name: low_required_education, dtype: int64

In [8]:
# feature contains money_in_title, money_in_description
currencies= ['$','£','¥','€']

# create a function to check if a string contains any currency symbol
def contains_currency(s):
    return any(currency in s for currency in currencies)

# reload raw data
raw_data = pd.read_csv("emscad_v1.csv")

# apply the function
cleaned_data['money_in_title'] = raw_data['title'].apply(contains_currency).astype(int)
cleaned_data['money_in_desc'] = raw_data['description'].apply(contains_currency).astype(int)

In [9]:
# check if text contains consecutive punctuations
punctuations = ['..','!!','??']

# create a function to check if a string contains consecutive punctuations
def contains_cons_punc(s):
    return any(punct in s for punct in punctuations)

# apply the function
cleaned_data['cons_punc_des'] = raw_data['description'].apply(contains_cons_punc).astype(int)
cleaned_data['cons_punc_title'] = raw_data['title'].apply(contains_cons_punc).astype(int)

In [10]:
# create some boolean variables
cleaned_data["has_short_company_profile"]= np.where(cleaned_data['company_profile'] < 10, 1, 0)
cleaned_data["has_short_requirements"]= np.where(cleaned_data['requirements'] < 10, 1, 0)

In [12]:
cleaned_data

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,...,function,fraudulent,in_balanced_dataset,low_required_education,money_in_title,money_in_desc,cons_punc_des,cons_punc_title,has_short_company_profile,has_short_requirements
0,Marketing Intern,"US, NY, New York",Marketing,NaN,137,155,170,1,0,1,...,Marketing,0,f,0,0,0,0,0,0,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,272,317,344,253,0,1,...,Customer Service,0,f,0,0,0,1,0,0,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,142,42,223,1,0,1,...,NaN,0,f,0,0,0,0,0,0,0
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,85,423,272,222,0,1,...,Sales,0,f,0,0,0,0,0,0,0
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,230,395,210,9,0,1,...,Health Care Provider,0,f,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,367,176,201,158,0,1,...,Sales,0,f,0,0,0,0,0,0,0
17876,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,287,227,173,249,0,1,...,Accounting/Auditing,0,f,0,0,0,0,0,0,0
17877,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,30,292,285,1,0,0,...,NaN,0,f,0,0,0,0,0,0,0
17878,Graphic Designer,"NG, LA, Lagos",NaN,NaN,1,70,101,37,0,0,...,Design,0,f,0,0,0,0,0,1,0


In [13]:
final_data = cleaned_data[['fraudulent', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions',
       'money_in_title', 'money_in_desc', 'cons_punc_des',
       'cons_punc_title', 'low_required_education',
       'has_short_company_profile', 'has_short_requirements']]
final_data

,fraudulent,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,money_in_title,money_in_desc,cons_punc_des,cons_punc_title,low_required_education,has_short_company_profile,has_short_requirements
0,0,137,155,170,1,0,1,0,0,0,0,0,0,0,0
1,0,272,317,344,253,0,1,0,0,0,1,0,0,0,0
2,0,142,42,223,1,0,1,0,0,0,0,0,0,0,0
3,0,85,423,272,222,0,1,0,0,0,0,0,0,0,0
4,0,230,395,210,9,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,0,367,176,201,158,0,1,1,0,0,0,0,0,0,0
17876,0,287,227,173,249,0,1,1,0,0,0,0,0,0,0
17877,0,30,292,285,1,0,0,0,0,0,0,0,0,0,0
17878,0,1,70,101,37,0,0,1,0,0,0,0,0,1,0


In [16]:
import seaborn as sns

#get correlations of each features in dataset
corrmat = final_data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(final_data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

TypeError: 'module' object is not callable

Result shows that the most important features are: has_short_company_profile, has_company_logo, company_profile, has_short_description, low_require_education, requirements, benefits, has_short_requirements, telecommuting.

## Selected features


In [76]:
selected_data = final_data

In [77]:
selected_data.to_csv('selected_data.csv', index=False)

## Train-test set

In [89]:
np.random.seed(26)
# Train_test
df_train = selected_data.sample(frac = 0.75)
df_test = selected_data.drop(df_train.index)

In [90]:
df_test['fraudulent'].value_counts()

0    4266
1     204
Name: fraudulent, dtype: int64

In [91]:
df_train['fraudulent'].value_counts()

0    12748
1      662
Name: fraudulent, dtype: int64

In [92]:
def over_sample(data):
    #Make a balanced dataset by oversampling
    data_Fraud = data[data['fraudulent']==1]
    data_NonFraud =  data[data['fraudulent']==0]    
    df_fraud_example = pd.concat([data_Fraud, data_Fraud, data_Fraud, data_Fraud, 
                                  data_Fraud, data_Fraud, data_Fraud, data_Fraud, 
                                  data_Fraud, data_Fraud, data_Fraud, data_Fraud, 
                                 data_Fraud, data_Fraud, data_Fraud, data_Fraud, 
                                 data_Fraud, data_Fraud, data_Fraud], axis = 0)
    data_balanced = pd.concat([data_NonFraud, df_fraud_example], axis = 0)
    data_balanced = data_balanced.sample(frac=1).reset_index(drop=True)
    return data_balanced

In [93]:
df_train_bal = over_sample(df_train)
df_train_bal['fraudulent'].value_counts()

0    12748
1    12578
Name: fraudulent, dtype: int64

In [94]:
df_test['fraudulent'].value_counts()

0    4266
1     204
Name: fraudulent, dtype: int64

In [95]:
df_train_bal.to_csv('df_train_bal.csv', index = False)
df_test.to_csv('df_test.csv', index = False)

In [96]:
df_train = pd.read_csv("df_train_bal.csv")
df_train

,description,has_short_company_profile,company_profile,requirements,benefits,has_company_logo,low_required_education,has_questions,telecommuting,has_short_requirements,money_in_title,money_in_desc,cons_punc_des,fraudulent
0,97,0,86,68,53,1,1,0,0,0,1,0,0,1
1,41,0,141,246,117,1,0,1,0,0,0,0,0,0
2,168,0,62,54,107,1,0,0,0,0,0,0,0,0
3,106,1,1,111,111,0,0,0,0,0,0,0,0,1
4,113,0,32,75,32,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25321,77,0,149,37,20,1,1,1,0,0,0,0,0,0
25322,90,0,90,129,61,1,0,1,0,0,0,0,0,0
25323,114,0,18,28,9,1,0,0,0,0,0,1,0,0
25324,645,0,46,154,37,1,0,0,0,0,0,0,0,1
